In [1]:
%%bash
mkdir datasetp
curl https://storage.googleapis.com/deepmind-media/research/Kinetics_700.zip --output dataset/kinects_700.zip

mkdir: não foi possível criar o diretório “dataset”: Arquivo existe
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 21.7M  100 21.7M    0     0  12.1M      0  0:00:01  0:00:01 --:--:-- 12.1M


In [2]:
%%bash
cd dataset
unzip kinects_700.zip

Archive:  kinects_700.zip


replace kinetics_700_readme.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [3]:
%%bash
cd dataset
unzip kinetics_700_train.zip
unzip kinetics_700_test.zip
unzip kinetics_700_val.zip

Archive:  kinetics_700_train.zip


replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [unzip kin]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [etics_700]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [_test.zip]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [{ENTER}]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [unzip kin]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [etics_700]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: error:  invalid response [_val.zip]
replace kinetics_700_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename:  NULL
(EOF or read error, treating as "[N]one" ...)


In [4]:
import pandas as pd
import argparse
import os
import subprocess
import pytube

In [5]:
REQUIRED_COLUMNS = ['label', 'youtube_id', 'time_start', 'time_end', 'split', 'is_cc']
TRIM_FORMAT = '%06d'
URL_BASE = 'https://www.youtube.com/watch?v='

VIDEO_EXTENSION = '.mp4'
VIDEO_FORMAT = 'mp4'

In [21]:
BASE_PATH = os.path.join('/', 'media' ,os.environ['USER'] ,'DATA','kinects')

TRAIN_VIDEOS_PATH = os.path.join(BASE_PATH, 'train')
VALIDATE_VIDEOS_PATH = os.path.join(BASE_PATH, 'validate')
TEST_VIDEOS_PATH = os.path.join(BASE_PATH, 'test')

In [7]:
if not os.path.exists(BASE_PATH):
    os.mkdir(BASE_PATH)
else:
    print(f'Path {BASE_PATH} already exists!')

In [9]:
def create_file_structure(path, folders_names):
    mapping = {}
    if not os.path.exists(path):
        os.mkdir(path)
    for name in folders_names:
        dir_ = os.path.join(path, name)
        if not os.path.exists(dir_):
            os.mkdir(dir_)
        mapping[name] = dir_
    return mapping

In [39]:
def trim(row, label_to_dir, test=False):
    label = row['label'] if not test else ''
    filename = row['youtube_id']
    time_start = row['time_start']
    time_end = row['time_end']
    
    output_path = label_to_dir['tmp'] if trim else label_to_dir[label]
    
    start = str(time_start)
    end = str(time_end - time_start)

    input_filename = os.path.join(output_path, filename + VIDEO_EXTENSION)
    output_filename = os.path.join(label_to_dir[label],
                                   filename + '_{}_{}'.format(start, end) + VIDEO_EXTENSION)

    if os.path.exists(output_filename):
        print('Already trimmed: ', filename)
    else:
        print('Start trimming: ', filename)
        command = 'ffmpeg -i "{input_filename}" ' \
                  '-ss {time_start} ' \
                  '-t {time_end} ' \
                  '-c:v libx264 -c:a copy -threads 1 ' \
                  '"{output_filename}"'.format(
                       input_filename=input_filename,
                       time_start=start,
                       time_end=end,
                       output_filename=output_filename
                   )
        try:
            subprocess.check_output(command, shell=True)
        except subprocess.CalledProcessError as e:
            print(f'Error {e} while trimming: ', filename)
            return False
        print('Finish trimming: ', filename)

In [40]:
def download_clip(row, label_to_dir, test=False):
    label = row['label'] if not test else ''
    filename = row['youtube_id']
    time_start = row['time_start']
    time_end = row['time_end']

    output_path = label_to_dir['tmp'] if trim else label_to_dir[label]

    if not os.path.exists(os.path.join(output_path, filename + VIDEO_EXTENSION)):
        print('Start downloading: ', filename)
        try:
            pytube.YouTube(URL_BASE + filename) \
                .streams \
                .filter(subtype=VIDEO_FORMAT) \
                .first() \
                .download(output_path, filename)
            print('Finish downloading: ', filename)
        except KeyError as e:
            print(f'Key Error {e}')
            return
        except Exception as e:
            print(f'Error in download video: {e}')
            return
    else:
        print('Already downloaded: ', filename)

## Donwload
Create a method to make a download and trim the videos. Repair that I delete the `tmp` file.

In [12]:
import shutil

In [46]:
def top_download(path_csv, target, to_download=5, test=False):
    links_dataframes = pd.read_csv(path_csv).head(to_download)
    
    if not test:
        folders_names = links_dataframes['label'].unique().tolist() + ['tmp']
        label_to_dir = create_file_structure(path=target, folders_names=folders_names)

        [download_clip(row, label_to_dir) for _, row in links_dataframes.iterrows()]
        [trim(row, label_to_dir) for _, row in links_dataframes.iterrows()]
        
        shutil.rmtree(label_to_dir['tmp'])
    else:
        folders_names = ['tmp', '']
        label_to_dir = create_file_structure(path=target, folders_names=folders_names)
        
        [download_clip(row, label_to_dir, test=True) for _, row in links_dataframes.iterrows()]
        [trim(row, label_to_dir, test=True) for _, row in links_dataframes.iterrows()]
        
        shutil.rmtree(label_to_dir['tmp'])

### Download of Train test
Make a test download of a five videos from train
___

The videos path is `dataset/kinetics_700_train.csv` and the target path is the variable `TRAIN_VIDEOS_PATH`.

In [24]:
head_download('dataset/kinetics_700_train.csv', TRAIN_VIDEOS_PATH)

Start downloading:  oJCxnjaCoyI
Finish downloading:  oJCxnjaCoyI
Processed 1 out of 0
Start downloading:  XtbeP16BJdQ
Finish downloading:  XtbeP16BJdQ
Processed 2 out of 0
Start downloading:  XXulvmGzw60
Finish downloading:  XXulvmGzw60
Processed 3 out of 0
Start downloading:  -3B32lodo2M
Error in download video: HTTP Error 403: Forbidden
Start downloading:  -7kbO0v4hag
Finish downloading:  -7kbO0v4hag
Processed 5 out of 0
Already trimmed:  oJCxnjaCoyI
Already trimmed:  XtbeP16BJdQ
Already trimmed:  XXulvmGzw60
Start trimming:  -3B32lodo2M
Error Command 'ffmpeg -i "/media/renato/DATA/kinects/train/tmp/-3B32lodo2M.mp4" -ss 59 -t 10 -c:v libx264 -c:a copy -threads 1 "/media/renato/DATA/kinects/train/abseiling/-3B32lodo2M_59_10.mp4"' returned non-zero exit status 1. while trimming:  -3B32lodo2M
Already trimmed:  -7kbO0v4hag


### Download of Validate test
Make a test download of a five videos from train
___

The videos path is `dataset/kinetics_700_val.csv` and the target path is the variable `VALIDATE_VIDEOS_PATH`.

In [25]:
head_download('dataset/kinetics_700_val.csv', VALIDATE_VIDEOS_PATH)

Start downloading:  ixq5OGYjjmA
Finish downloading:  ixq5OGYjjmA
Processed 1 out of 0
Start downloading:  j2_AKSjeQy0
Error in download video: HTTP Error 403: Forbidden
Start downloading:  J_faWIyCzdc
Finish downloading:  J_faWIyCzdc
Processed 3 out of 0
Start downloading:  Kbmv94BqZzU
Finish downloading:  Kbmv94BqZzU
Processed 4 out of 0
Start downloading:  LFMlCcsLdKA
Error in download video: HTTP Error 403: Forbidden
Start trimming:  ixq5OGYjjmA
Finish trimming:  ixq5OGYjjmA
Start trimming:  j2_AKSjeQy0
Error Command 'ffmpeg -i "/media/renato/DATA/kinects/validate/tmp/j2_AKSjeQy0.mp4" -ss 1 -t 10 -c:v libx264 -c:a copy -threads 1 "/media/renato/DATA/kinects/validate/abseiling/j2_AKSjeQy0_1_10.mp4"' returned non-zero exit status 1. while trimming:  j2_AKSjeQy0
Start trimming:  J_faWIyCzdc
Finish trimming:  J_faWIyCzdc
Start trimming:  Kbmv94BqZzU
Finish trimming:  Kbmv94BqZzU
Start trimming:  LFMlCcsLdKA
Error Command 'ffmpeg -i "/media/renato/DATA/kinects/validate/tmp/LFMlCcsLdKA.mp

### Download of Test
Make a test download of a five videos from test
___

The videos path is `dataset/kinetics_700_test.csv` and the target path is the variable `TEST_VIDEOS_PATH`.

In [48]:
top_download('dataset/kinetics_700_test.csv', TEST_VIDEOS_PATH, to_download=8, test=True)

Start downloading:  6dEpI75FOeo
Finish downloading:  6dEpI75FOeo
Start downloading:  15H3EqaHVi0
Error in download video: HTTP Error 403: Forbidden
Start downloading:  d6Ko4hm8M8E
Finish downloading:  d6Ko4hm8M8E
Start downloading:  -SakeFNtM0s
Finish downloading:  -SakeFNtM0s
Start downloading:  at3mGS-FAVg
Finish downloading:  at3mGS-FAVg
Start downloading:  0-qpI81QREc
Finish downloading:  0-qpI81QREc
Start downloading:  G5naKwYvSFM
Finish downloading:  G5naKwYvSFM
Start downloading:  JlaQtvL6tgQ
Finish downloading:  JlaQtvL6tgQ
Already trimmed:  6dEpI75FOeo
Start trimming:  15H3EqaHVi0
Error Command 'ffmpeg -i "/media/renato/DATA/kinects/test/tmp/15H3EqaHVi0.mp4" -ss 8 -t 10 -c:v libx264 -c:a copy -threads 1 "/media/renato/DATA/kinects/test/15H3EqaHVi0_8_10.mp4"' returned non-zero exit status 1. while trimming:  15H3EqaHVi0
Already trimmed:  d6Ko4hm8M8E
Already trimmed:  -SakeFNtM0s
Already trimmed:  at3mGS-FAVg
Start trimming:  0-qpI81QREc
Finish trimming:  0-qpI81QREc
Start trimm